# Job ETL: Raw -> Silver


Notebook responsável por **extrair os dados da camada Raw**, **realizar limpezas, padronizações e remoção de outliers**, e **carregar os resultados tratados na camada Silver**.  
Execute as células na ordem apresentada e ajuste apenas os parâmetros indicados.



---
## 1. Congurações Iniciais

### 1.1 Importação das bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, split
from pyspark.sql.types import DoubleType, IntegerType
import os
import shutil


### 1.2 Configuração do Ambiente Hadoop para Windows

Para permitir o correto funcionamento do Spark no Windows, é necessária a configuração do ambiente Hadoop local, apontando para os binários de suporte (`winutils.exe` e `hadoop.dll`).

In [2]:
os.environ['HADOOP_HOME'] = 'C:\\hadoop'
os.environ['PATH'] = f"{os.environ['HADOOP_HOME']}\\bin;{os.environ['PATH']}"


### 1.3 Limpeza de Diretório Temporário

Antes da execução, o diretório temporário do Spark é limpo e recriado, garantindo que não haja resíduos de execuções anteriores.

In [3]:
temp_dir = "C:\\spark-temp"
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
os.makedirs(temp_dir)


### 1.4 Inicialização da SparkSession 

A `SparkSession` é criada com parâmetros específicos para definir os diretórios locais e de warehouse dentro do diretório temporário criado anteriormente.  
Essas configurações evitam erros de caminho e melhoram a compatibilidade em ambientes Windows.

In [4]:
temp_dir_uri = temp_dir.replace('\\', '/')


spark = SparkSession.builder \
    .appName("Formula1_Raw_to_Silver_Outliers") \
    .config("spark.sql.warehouse.dir", f"{temp_dir_uri}/warehouse") \
    .config("spark.local.dir", f"{temp_dir_uri}/local") \
    .getOrCreate()


### 1.5 Iniciação dos Caminhos das Camadas

Os diretórios de origem (Raw) e destino (Silver) são definidos.  
O caminho de destino é criado automaticamente, caso não exista.

In [5]:
base_path = os.path.join("C:", os.sep, "Users", "julii", "OneDrive", "Documentos", "formula1-analytics", "Data Layer")
raw_path = os.path.join(base_path, "raw", "dados_originais")
silver_path = os.path.join(base_path, "silver", "dados_limpos")

os.makedirs(silver_path, exist_ok=True)
print(f"Diretório de origem (RAW): {raw_path}")
print(f"Diretório de destino (SILVER): {silver_path}\n")

Diretório de origem (RAW): C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\raw\dados_originais
Diretório de destino (SILVER): C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos




### 1.6 Leitura dos Dados da Camada RAW

Todos os arquivos CSV da camada Raw são carregados em DataFrames Spark.  
O parâmetro `nullValue="\\N"` garante que valores nulos sejam interpretados corretamente.

In [6]:
print(" Lendo arquivos da camada RAW com tratamento de nulos (\\N)... \n")

df_lap_times = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "lap_times.csv"), header=True, inferSchema=True)
df_results = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "results.csv"), header=True, inferSchema=True)
df_races = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "races.csv"), header=True, inferSchema=True)
df_drivers = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "drivers.csv"), header=True, inferSchema=True)
df_constructors = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "constructors.csv"), header=True, inferSchema=True)
df_status = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "status.csv"), header=True, inferSchema=True)
df_pit_stops = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "pit_stops.csv"), header=True, inferSchema=True)

print(" Arquivos RAW carregados com sucesso!\n")

 Lendo arquivos da camada RAW com tratamento de nulos (\N)... 

 Arquivos RAW carregados com sucesso!



---
## 2. Remoção de Outliers

Nesta etapa, com base no método **IQR (Interquartile Range)**, ocorre a padronização dos dados e a eliminação dos outliers.

### 2.1 Tabela Corridas


In [7]:
races_silver = df_races.select(
    col("raceId").alias("id_corrida"),
    col("year").alias("ano"),
    col("round").alias("rodada"),
    col("name").alias("nome_corrida")
)

### 2.2 Tabela Pilotos

In [8]:
drivers_silver = df_drivers.select(
    col("driverId").alias("id_piloto"),
    col("forename").alias("primeiro_nome_piloto"),
    col("surname").alias("sobrenome_piloto")
)

### 2.2 Tabela Equipes

In [9]:
constructors_silver = df_constructors.select(
    col("constructorId").alias("id_equipe"),
    col("name").alias("nome_equipe")
)

### 2.3 Tabela Pit Stops
A coluna `duration` é convertida para segundos, tratando valores com ou sem formato “min:seg”.  
Após isso, são removidos outliers que ultrapassam os limites definidos pelo IQR.

In [10]:
pit_stops_cleaned = df_pit_stops.withColumn(
    "duracao_parada_seg",
    when(
        col("duration").contains(":"), 
        split(col("duration"), ":").getItem(0).cast("double") * 60 +
        split(col("duration"), ":").getItem(1).cast("double")
    ).otherwise(
        col("duration").cast("double")
    )
)

pit_stops_silver = pit_stops_cleaned.select(
    col("raceId").alias("id_corrida"),
    col("driverId").alias("id_piloto"),
    "duracao_parada_seg"
).na.drop(subset=["duracao_parada_seg"])

print("    → Removendo outliers de 'pit_stops' (duração)...")

    → Removendo outliers de 'pit_stops' (duração)...


### 2.4 Tabela Status

In [11]:
status_silver = df_status.select(
    col("statusId").alias("id_status"),
    col("status").alias("descricao_status")
)

### 2.5 Tabela Resultados

In [12]:
results_silver = df_results.select(
    col("raceId").alias("id_corrida"),
    col("driverId").alias("id_piloto"),
    col("constructorId").alias("id_equipe"),
    col("statusId").alias("id_status")
)

### 2.6 Tabela Tempos de Volta (Fato)

In [13]:
lap_times_fact = df_lap_times.select(
    col("raceId").alias("id_corrida"),
    col("driverId").alias("id_piloto"),
    col("lap").alias("volta"),
    col("position").alias("posicao_na_volta"),
    expr("try_cast(milliseconds as integer)").alias("tempo_volta_ms")
).na.drop(subset=["tempo_volta_ms"])


---
## 3. Carga em Formato Parquet

Os DataFrames processados são salvos no formato **Parquet**, o que melhora o desempenho e a compressão, facilitando consultas posteriores.

In [14]:
# --- COLOQUE TUDO NA MESMA CÉLULA ---

print(" Salvando todas as tabelas na camada SILVER em formato PARQUET...\n")

# 1. Definição do dicionário
silver_tables = {
    "races": races_silver,
    "drivers": drivers_silver,
    "constructors": constructors_silver,
    "pit_stops": pit_stops_silver,
    "status": status_silver,
    "results": results_silver,
    "lap_times_fact": lap_times_fact
}

# 2. Loop para salvar os dados
for name, df in silver_tables.items():
    output_path = os.path.join(silver_path, name)
    df.coalesce(1).write.mode("overwrite").parquet(output_path)
    print(f"  → Dados salvos em Parquet: {output_path}")

print("\n Camada Silver gerada com sucesso!\n")

 Salvando todas as tabelas na camada SILVER em formato PARQUET...

  → Dados salvos em Parquet: C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos\races
  → Dados salvos em Parquet: C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos\drivers
  → Dados salvos em Parquet: C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos\constructors
  → Dados salvos em Parquet: C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos\pit_stops
  → Dados salvos em Parquet: C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos\status
  → Dados salvos em Parquet: C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos\results
  → Dados salvos em Parquet: C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos\lap_times_fact

 Camada Silver gerada com sucesso!



---
## 4. Validação

Por fim, é exibido o schema e uma amostra da tabela fato principal, confirmando o sucesso do processo de transformação.

In [15]:
print(" Validando a tabela fato principal 'lap_times_fact':\n")
lap_times_fact.printSchema()
lap_times_fact.show(10)

print("\n Job ETL (Raw → Silver) finalizado com sucesso!")

 Validando a tabela fato principal 'lap_times_fact':

root
 |-- id_corrida: integer (nullable = true)
 |-- id_piloto: integer (nullable = true)
 |-- volta: integer (nullable = true)
 |-- posicao_na_volta: integer (nullable = true)
 |-- tempo_volta_ms: integer (nullable = true)

+----------+---------+-----+----------------+--------------+
|id_corrida|id_piloto|volta|posicao_na_volta|tempo_volta_ms|
+----------+---------+-----+----------------+--------------+
|       841|       20|    1|               1|         98109|
|       841|       20|    2|               1|         93006|
|       841|       20|    3|               1|         92713|
|       841|       20|    4|               1|         92803|
|       841|       20|    5|               1|         92342|
|       841|       20|    6|               1|         92605|
|       841|       20|    7|               1|         92502|
|       841|       20|    8|               1|         92537|
|       841|       20|    9|               1|     